## set DB

- Chroma DB 를 통해 VectorStore 를 구축한다.
- 벡터 저장소 생성 from_texts

In [1]:
import torch
from load_embedding import load_embedding
from pdf_preprocessing import LegalText
from langchain_core.documents import Document
from langchain_chroma import Chroma
from glob import glob
from pdf_preprocessing import LegalText

# filenames = glob('/Volumes/MINDB/24년/SW아카데미/LangEyE/App/src/crawling/files/laws/*.pdf')
filenames = glob('/workspace/LangEyE/App/src/crawling/files/laws/*.pdf')

# print(filenames[0])
# LegalText(filenames[0]).documents

law_docs = []
for filename in filenames:
    law_docs.extend(LegalText(filename).documents)
print(len(law_docs))

501


In [2]:
if torch.cuda.is_available():
    device = torch.device("cuda")
elif torch.backends.mps.is_available():
    device = torch.device("mps")
else:
    device = torch.device("cpu")

DB_PATH = "./chroma_db"

db = Chroma.from_documents(
    documents=law_docs, embedding=load_embedding(device),
    persist_directory=DB_PATH,
    collection_name="my_db"
)

/workspace/LangEyE/App/models/RAG/load_embedding.py:23: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(
/usr/local/lib/python3.10/dist-packages/transformers/utils/hub.py:128: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


In [3]:
db.get()

{'ids': ['688913f5-3a4f-4802-90df-d96d3c233aea',
  '6ed23f93-e486-4099-9038-fbc28a6e701a',
  '292c8961-135c-45b2-8367-3ca24a26fe35',
  'ab3306f0-f7c4-421a-9f1c-7a224af1b2f5',
  '07cd7fe6-9e3c-4418-99b3-cd2d25e87dd4',
  '323c0e58-a0fa-4cf1-bd58-256a069f4853',
  '3041c77b-edf3-4153-81a0-8179ed905089',
  'c7b082b2-d525-4e73-a0c3-7fad674b9ac6',
  '09c4d1b5-2839-4e4a-92f1-9506133c0d35',
  'db07a37f-1426-48c2-9a27-2d23c8420b93',
  'd4f55f4b-6fe5-4583-831c-4fff89c7c395',
  '7cc59b43-4a2c-4c21-a93d-99e631376f41',
  '394bdaf5-63c8-4678-9f67-931b25225bef',
  '270e3fca-41d8-4d0d-9dfd-a651bc71f048',
  '2c90396f-236c-45d1-b8c7-0e52da94684e',
  '6a0d7c2e-36d5-4c99-8374-a50ab7401c38',
  '146bc57f-be8d-4306-8c92-6e2dea846558',
  '7cb26b30-c16c-4fa8-b992-aece67283a3c',
  '79a611de-50fd-4af5-876b-be3dd521fd00',
  '2540fd16-1ccd-4a2b-8224-2323efe1fa6e',
  '50818182-0402-43fe-9536-2e0aa35908ea',
  '7a0e84c8-0581-4342-b6f9-6cc78a450aae',
  '9e5cbc69-624f-40c6-949f-2385f16bed3c',
  'fe7ec1be-af22-4a5b-9b98-

## DB 로드하기

In [ ]:
DB_PATH = "./chroma_db"
if torch.cuda.is_available():
    device = torch.device("cuda")
elif torch.backends.mps.is_available():
    device = torch.device("mps")
else:
    device = torch.device("cpu")

db = Chroma(
    persist_directory=DB_PATH,
    embedding_function=load_embedding(device),
    collection_name="my_db"
)

### 유사도 검색

`similarity_search` 메서드는 Chroma 데이터베이스에서 유사도 검색을 수행합니다. 이 메서드는 주어진 쿼리와 가장 유사한 문서들을 반환합니다.

**매개변수**

- `query` (str): 검색할 쿼리 텍스트
- `k` (int, 선택적): 반환할 결과의 수. 기본값은 4입니다.
- `filter` (Dict[str, str], 선택적): 메타데이터로 필터링. 기본값은 None입니다.

**참고**

- `k` 값을 조절하여 원하는 수의 결과를 얻을 수 있습니다.
- `filter` 매개변수를 사용하여 특정 메타데이터 조건에 맞는 문서만 검색할 수 있습니다.
- 이 메서드는 점수 정보 없이 문서만 반환합니다. 점수 정보도 필요한 경우 `similarity_search_with_score` 메서드를 직접 사용하세요.

**반환값**

- `List[Document]`: 쿼리 텍스트와 가장 유사한 문서들의 리스트

- query 에서 Pattern 을 찾아서 filter 를 적용한다.

In [ ]:
# filter 사용
db.similarity_search(
    "TF IDF 에 대하여 알려줘", filter={"source": "data/nlp-keywords.txt"}, k=2
)